In [2]:
import gym
import copy
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time 
from MCTS import MCTS
from model import PolicyNet

In [2]:
t = time.time()
1+1
print(time.time()-t)

1587119114.9983168

In [3]:
GRID = 7

In [4]:
go_env = gym.make('gym_go:go-v0', size=GRID, reward_method='real')

In [5]:
first_action = 1

In [6]:
go_env.turn()

0

In [9]:
L = [(1, 2, 3), (1, 2, 3), (1, 2, 3)]
np.mean(L, axis=0)

array([2., 2., 2.])

In [7]:
go_env1 = copy.copy(go_env)

In [8]:
state, reward, done, info = go_env.step(first_action)
go_env.render('terminal')

0   1   2   3   4   5   6
  -----------------------------
0 | . | B | . | . | . | . | . |
  -----------------------------
1 | . | . | . | . | . | . | . |
  -----------------------------
2 | . | . | . | . | . | . | . |
  -----------------------------
3 | . | . | . | . | . | . | . |
  -----------------------------
4 | . | . | . | . | . | . | . |
  -----------------------------
5 | . | . | . | . | . | . | . |
  -----------------------------
6 | . | . | . | . | . | . | . |
  -----------------------------
	Turn: W, Last Turn Passed: False, Game Over: 0
	Black Area: 49.0, White Area: 0.0



In [9]:
go_env.game_ended()

0

In [10]:
mod = PolicyNet(GRID, 5, GRID**2+1)
Mcts = MCTS(mod, 300)

TypeError: __init__() missing 1 required positional argument: 'reg'

In [11]:
"""
dataset = tf.data.Dataset.from_tensors(HIST)
dataset = dataset.batch(batch_size, drop_remainder=True).repeat()
iterator = list(dataset.as_numpy_iterator())
for epoch in range(epochs):
    for batch in iterator:
        x, y = batch
        
        board, ind = x # Inputs: board as a numpy array and passing indicator
        pie, z = y # Labels: Target probability distribution and winning indicator

        with tf.GradientTape() as tape:
            loss = MctsT.guidingNet.get_loss(board, ind, pie, z, training=True) # compute the loss
            variables = MctsT.guidingNet.trainable_variables # get the trainable variables
            gradients = tape.gradients(loss, variables) # compute the gradients of the loss wrt those variables

        optimizer.apply_gradients(zip(gradients, variables)) # update the trainable weights
"""

'\ndataset = tf.data.Dataset.from_tensors(HIST)\ndataset = dataset.batch(batch_size, drop_remainder=True).repeat()\niterator = list(dataset.as_numpy_iterator())\nfor epoch in range(epochs):\n    for batch in iterator:\n        x, y = batch\n        \n        board, ind = x # Inputs: board as a numpy array and passing indicator\n        pie, z = y # Labels: Target probability distribution and winning indicator\n\n        with tf.GradientTape() as tape:\n            loss = MctsT.guidingNet.get_loss(board, ind, pie, z, training=True) # compute the loss\n            variables = MctsT.guidingNet.trainable_variables # get the trainable variables\n            gradients = tape.gradients(loss, variables) # compute the gradients of the loss wrt those variables\n\n        optimizer.apply_gradients(zip(gradients, variables)) # update the trainable weights\n'

In [12]:
def to_numpy(state): 
    # Takes a state as given by env.step and return the numpy array for the board and the passing indicator
    P0 = state[0] # 2D array of the pieces of P0
    P1 = state[1] # 2D array of the pieces of P1
    turn = state[2][0] # =0 if it's P0's turn =1 if P1's
    prev_pass =  state[4][0] # Indicicator for whether the previous turn was a pass
    if turn == 0: # Board representation, 1 for the current player's pieces -1 for the other's
        board = P0 - P1
    else :
        board = P1 - P0
    return board, prev_pass

In [16]:
lr=5e-3
nb_data_gather_games=100
max_time_step = 100
temperature = 1
c = 1

In [17]:

env_state = gym.make('gym_go:go-v0', size=GRID, reward_method='real')


MctsD = MCTS(PolicyNet(GRID, 5, GRID**2+1, reg=0.1), 300)
MctsT = MCTS(PolicyNet(GRID, 5, GRID**2+1, reg=0.1), 300)

optimizer = tf.optimizers.Adam(lr)
while True:
    HIST = []
    for game_ix in range(nb_data_gather_games):
        state = env_state.reset()
        t=0
        while t<max_time_step : #and avg_v > min_val:
            
            pie = MctsD.search(env_state, temperature, c)
            HIST.append((to_numpy(state), pie, env_state.turn()))
            action = np.random.choice(GRID**2+1, p=pie)
            state, _, _, _ = env_state.step(action)
            if env_state.game_ended():
                break
            t += 1
        z = 0 if env.get_winning()==1 else 1
    HIST = [((board, ind), (pie, z==turn_id)) for (board, ind), pie, turn_id in HIST]
    
    dataset = tf.data.Dataset.from_tensors(HIST)
    dataset = dataset.batch(batch_size, drop_remainder=True).repeat()
    iterator = list(dataset.as_numpy_iterator())
    for epoch in range(epochs):
        for batch in iterator:
            x, y = batch
            
            board, ind = x # Inputs: board as a numpy array and passing indicator
            pie, z = y # Labels: Target probability distribution and winning indicator

            with tf.GradientTape() as tape:
                loss = MctsT.guidingNet.get_loss(board, ind, pie, z, training=True) # compute the loss
                variables = MctsT.guidingNet.trainable_variables # get the trainable variables
                gradients = tape.gradients(loss, variables) # compute the gradients of the loss wrt those variables

            optimizer.apply_gradients(zip(gradients, variables)) # update the trainable weights
        
        
    t_score = 0
    for i in range(tournament_len):
        # Tournamnet to see would wins in a MCTS battle between the training net and the data generation net
        env_state.reset()
        A = ['t', 'd'] if i%2 else ['d', 't'] # to alternate who starts
        D = {'t' : MctsT,  'd' : MctsD}
        t=0
        while t<max_time_step : # and avg_v > min_val:
            for k in A:
                pie = D[k].search(env_state, temperature, c)
                action = np.random.choice(GRID_SIZE**2+1, p=pie)
                env_state.step(action)
                if env_state.game_ended():
                    break
                t+=1
            if env_state.game_ended():
                    break

        z = env_state.get_winning()

        if z == 1:
            # A[0] won cuz he started
            if A[0] == 't':
                t_score += 1
            else :
                t_score -= 1
        elif z == -1:
            # A[1] won cuz he didnt start
            if A[1] == 't':
                t_score += 1
            else :
                t_score -= 1
    
    # Here t_score represent the game_won-game_lost for the training network 
    if t_score>0 : 
        # If the training net won more games that means it's better so we use it for the next MCTS data gathering
        MctsD.guidingNet = copy.copy(MctsT.guidingNet)





False


ValueError: attempt to get argmax of an empty sequence

In [15]:
self_play()

NameError: name 'self_play' is not defined